In [0]:
# - Jason Votaw 2025
# You need to run these to install the required packages (only once)
# %pip install pandas openpyxl
# %pip install faker

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def generate_provider_kpi_data(measures_with_flags, benchmark_values, trends, clinic_providers, weeks_back=52):
    data = []
    today = datetime.today()
    last_sunday = today - timedelta(days=today.weekday() + 1)
    date_range = [last_sunday - timedelta(weeks=w) for w in range(weeks_back)][::-1]
    
    # Define low-performing providers (2 providers across different clinics)
    low_performers = ["Dr. Michael Chen", "Dr. Rachel Martinez"]
    
    for clinic, providers in clinic_providers.items():
        for provider in providers:
            for measure, lower_is_better in measures_with_flags.items():
                benchmark = benchmark_values[measure]
                trend = trends.get(measure, 'neutral')
                
                # Start near the benchmark ±15% for more provider variation
                start_value = min(max(benchmark * random.uniform(0.85, 1.15), 0), 1)
                current_value = start_value
                
                # Adjust performance bias
                if provider in low_performers:
                    # Low performers: worse performance and negative trend
                    performance_modifier = 0.85  # Start 15% below benchmark
                    trend_drift = -0.008 if trend == 'up' else 0.008 if trend == 'down' else -0.003
                elif clinic == "East Clinic":
                    # East Clinic slightly worse performance
                    performance_modifier = 0.95
                    trend_drift = -0.003 if trend == 'up' else 0.003 if trend == 'down' else 0.0
                else:
                    # Normal performance
                    performance_modifier = 1.0
                    trend_drift = 0.005 if trend == 'up' else -0.005 if trend == 'down' else 0.0
                
                # Apply performance modifier to starting value
                current_value = min(max(current_value * performance_modifier, 0), 1)
                
                # Set consistent base denominator for this provider-measure combination
                base_denominator = int(random.uniform(800, 1500))
                
                for date in date_range:
                    noise = random.uniform(-0.025, 0.025)  # Slightly more variation at provider level
                    delta = trend_drift + noise
                    current_value = min(max(current_value + delta, 0), 1)
                    
                    # Very consistent denominators with minimal week-to-week variation (±2%)
                    denominator_variation = random.uniform(0.98, 1.02)
                    denominator = int(base_denominator * denominator_variation)
                    numerator = int(round(current_value * denominator))
                    
                    data.append({
                        'Clinic': clinic,
                        'Provider': provider,
                        'MeasureName': measure,
                        'MeasureDate': date.strftime('%Y-%m-%d'),
                        'LowerIsBetter': lower_is_better,
                        'Numerator': numerator,
                        'Denominator': denominator,
                        'MeasureValue': round(numerator / denominator, 4),
                        'Benchmark': round(benchmark, 4)
                    })
    
    return pd.DataFrame(data)

# Measures and flags (0 = higher is better, 1 = lower is better)
measures = {
    'Hypertension Control': 0,
    'Senior Readmissions': 1,
    'Diabetes Control': 0,
    'Breast Cancer Screening': 0,
    'Colon Cancer Screening': 0,
    'Referral Turnaround Time': 1,
    'Depression Screening': 0,
    'Depression Remission': 0,
    # New quality metrics
    'Patient Satisfaction': 0,
    'Statin Therapy for CVD Prevention': 0,
    'Adult Immunization Status': 0,
    'Medication Adherence': 0
}

# Benchmarks (industry standard targets)
benchmarks = {
    'Hypertension Control': 0.72,
    'Senior Readmissions': 0.12,
    'Diabetes Control': 0.50,
    'Breast Cancer Screening': 0.82,
    'Colon Cancer Screening': 0.80,
    'Referral Turnaround Time': 0.90,
    'Depression Screening': 0.75,
    'Depression Remission': 0.60,
    # New benchmarks
    'Patient Satisfaction': 0.85,  # 85% satisfied or very satisfied
    'Statin Therapy for CVD Prevention': 0.75,  # 75% of eligible patients on statin therapy
    'Adult Immunization Status': 0.70,  # 70% up-to-date on recommended vaccines
    'Medication Adherence': 0.80  # 80% adherence rate (PDC ≥ 0.8)
}

# Trends (up = improving, down = declining, neutral = stable)
trends = {
    'Hypertension Control': 'up',
    'Senior Readmissions': 'down',
    'Diabetes Control': 'up',
    'Breast Cancer Screening': 'up',
    'Colon Cancer Screening': 'up',
    'Referral Turnaround Time': 'down',
    'Depression Screening': 'up',
    'Depression Remission': 'up',
    # New trends
    'Patient Satisfaction': 'up',
    'Statin Therapy for CVD Prevention': 'up',
    'Adult Immunization Status': 'up',
    'Medication Adherence': 'neutral'
}

# Clinics with 6 providers each
clinic_providers = {
    'North Clinic': [
        'Dr. Sarah Johnson',
        'Dr. Michael Chen',  # Low performer
        'Dr. Jennifer Williams',
        'Dr. David Thompson',
        'Dr. Lisa Anderson',
        'Dr. Robert Garcia'
    ],
    'South Clinic': [
        'Dr. Amanda Taylor',
        'Dr. Christopher Lee',
        'Dr. Michelle Davis',
        'Dr. Kevin Brown',
        'Dr. Rachel Martinez',  # Low performer
        'Dr. Steven Wilson'
    ],
    'East Clinic': [
        'Dr. Nicole Rodriguez',
        'Dr. Daniel Moore',
        'Dr. Ashley Clark',
        'Dr. Matthew Lewis',
        'Dr. Jessica Walker',
        'Dr. Andrew Hall'
    ],
    'West Clinic': [
        'Dr. Emily Young',
        'Dr. Brandon King',
        'Dr. Samantha Wright',
        'Dr. Tyler Lopez',
        'Dr. Megan Hill',
        'Dr. Jonathan Green'
    ]
}

# Generate and save
df_provider_kpi = generate_provider_kpi_data(measures, benchmarks, trends, clinic_providers)
df_provider_kpi.to_csv("provider_kpi_data.csv", index=False)

print("CSV saved: provider_kpi_data.csv")
print(f"Total records generated: {len(df_provider_kpi)}")
print(f"Total providers: {sum(len(providers) for providers in clinic_providers.values())}")
print(f"Measures per provider: {len(measures)}")
print(f"Weeks of data: 52")
print("\nLow-performing providers:")
print("- Dr. Michael Chen (North Clinic)")
print("- Dr. Rachel Martinez (South Clinic)")
print("\nColumns in dataset:")
print("- Clinic: Clinic name")
print("- Provider: Provider name") 
print("- MeasureName: Quality measure")
print("- MeasureDate: Week ending date")
print("- LowerIsBetter: 0=higher better, 1=lower better")
print("- Numerator: Count of patients meeting criteria")
print("- Denominator: Total eligible patients")
print("- MeasureValue: Performance rate (numerator/denominator)")
print("- Benchmark: Industry standard target")

CSV saved: provider_kpi_data.csv
Total records generated: 14976
Total providers: 24
Measures per provider: 12
Weeks of data: 52

Low-performing providers:
- Dr. Michael Chen (North Clinic)
- Dr. Rachel Martinez (South Clinic)

Columns in dataset:
- Clinic: Clinic name
- Provider: Provider name
- MeasureName: Quality measure
- MeasureDate: Week ending date
- LowerIsBetter: 0=higher better, 1=lower better
- Numerator: Count of patients meeting criteria
- Denominator: Total eligible patients
- MeasureValue: Performance rate (numerator/denominator)
- Benchmark: Industry standard target
